# Multi-Class Classification - Language Classification

This notebook implements the method presented in Goldberg's [2017] book "Neural Network Methods for Natural Language Processing". It shows the steps you need to go through in order to successfully train a classifier, and it should also, so I hope, illustrate the notational differences between Goldberg and standard machine learning literature.

$NOTE$: There is no cross-validation etc. to find optimal parameters. This is simply to show how multi-class classification works. This will be part of a tutorial session and all other concepts will be explained there.

Author: Phillip Ströbel

## Getting and cleaning the data

The data consists of downloaded Wikipedia articles (see `urls.txt`) in German, English, French, Spanish, Italian and Finnish (instead of "O" in Goldberg). The data is in HTML, so we need to some preprocessing to get the text out of it. We also restrict ourselfes to the characters from a to z in the alphabet (as described in Goldberg). In this fashion, we get rid of all the Umlauts (ä, ö, ü) and all other characters with diacritics (as, e.g., the é or ç in French). Note however, that if these characters ocurring in bigrams would probably be good features. In some way, we still keep the information "special character" by not fully deleting the character, but by replacing it by the dollar sign "\$". Furthermore, we replace all punctuation marks and digits by dollar signs as well. As such, all special characters, digits, and punctuation marks are mapped to $. The space will be replaced by an underscore "\_". We then represent each langauge by 28 characters, as is suggested by Goldberg.

### Cleaning HTML
We first strip the HTML to get only the text of the Wikipedia page.

#### Get the html files

In [2]:
import re
import numpy as np
from bs4 import BeautifulSoup
from urllib.request import urlopen
from collections import defaultdict

seed = np.random.seed(seed=200)  # set a seed for random, so results are reproducible

article_dict = defaultdict(lambda: defaultdict(str))

regex = r'[\n ]{2,}'
pattern = re.compile(regex)

urls = open('urls.txt', 'r').readlines()

for index, url in enumerate(urls):
    language = url[8:10]
    doc_id = 'doc_%d' % index
    html = urlopen(url.strip()).read()    
    soup = BeautifulSoup(html, 'html.parser')
    raw = soup.body.get_text()  # only get text from the text body (this excludes headers and should exclude navigation bars)
    raw = re.sub(pattern, ' ', raw)  # replace multiple breaks and spaces by only one space
    raw = re.sub(r'\n', ' ', raw)  # replace every line break with a space
    article_dict[language][doc_id] = raw.lower()  # assign each text to its language and lower all uppercase characters

### Preprocessing --> prepare the text
replace special characters and digits

In [3]:
preprocessed_dict = defaultdict(lambda: defaultdict(str))

abc = r'[a-z]'
abc_pattern = re.compile(abc)

for lang, doc in article_dict.items():
    for doc, text in doc.items():
        for char in text:
            if re.match(abc_pattern, char):
                preprocessed_dict[lang][doc] += char
            elif re.match(' ', char):
                preprocessed_dict[lang][doc] += '_'
            else:
                preprocessed_dict[lang][doc] += '$'

### Count bigrams --> Feature extraction

The distribution of bigrams will be our only feature. We could extend this by taking into account other n-grams.

In [4]:
charset = 'abcdefghijklmnopqrstuvwxyz$_'  # define the character set we want to use

In [5]:
from itertools import combinations_with_replacement, permutations

def bigrams(text):
    """
    Function to extract bigrams from text and calculate their distribution
    :param text: text string
    :return: dictionary containing bigrams as keys, and the normalised count as values
    """
    combs = combinations_with_replacement(charset, 2)
    perms = permutations(charset, 2)
    bigram_dict = dict()
    
    for comb in set(list(combs) + list(perms)):
        bigram_dict[''.join(comb)] = 0
        
    doc_length = len(text)
    
    for index in range(0, len(text)-1):
        bigram = text[index] + text[index+1]
        bigram_dict[bigram] += 1
                
    for bigram, count in bigram_dict.items():
        bigram_dict[bigram] = count/doc_length

    return bigram_dict              

### Put data into pandas dataframe
The pandas dataframe allows us to conveniently represent all the data we need in one table. So let's do this. But first we need to extract the features.

In [6]:
bigram_dict_full = defaultdict(lambda: defaultdict(dict))

for lang, doc in preprocessed_dict.items():
    for doc, text in sorted(doc.items()):
        bigram_dict = bigrams(text)
        bigram_dict_full[lang][doc] = bigram_dict

In [7]:
import pandas as pd

col_names = ['y'] + sorted(bigram_dict_full['en']['doc_0'].keys())
my_df = dict()

for col in col_names:
    my_df[col] = list()
    
df = pd.DataFrame(my_df)

for lang, doc in bigram_dict_full.items():
    for key, value in doc.items():
        df_obj = value
        df_obj['y'] = lang
        df = df.append(df_obj, ignore_index=True)
        
df.head()
        

,$$,$_,$a,$b,$c,$d,$e,$f,$g,$h,...,zq,zr,zs,zt,zu,zv,zw,zx,zy,zz
0,0.100732,0.036889,0.000292,0.000222,0.000332,0.000163,0.000239,0.000181,0.000099,0.000152,...,0.000000,0.000006,0.000006,0.0,0.000023,0.000000,0.000000,0.0,0.000000,0.000006
1,0.062182,0.028909,0.000513,0.000322,0.000520,0.000356,0.000629,0.000192,0.000185,0.000171,...,0.000007,0.000007,0.000000,0.0,0.000027,0.000000,0.000000,0.0,0.000014,0.000021
2,0.072008,0.031248,0.000444,0.000519,0.000497,0.000187,0.000340,0.000246,0.000093,0.000179,...,0.000000,0.000004,0.000000,0.0,0.000030,0.000004,0.000004,0.0,0.000011,0.000000
3,0.075458,0.035249,0.000400,0.000466,0.000453,0.000268,0.000303,0.000224,0.000312,0.000189,...,0.000000,0.000004,0.000000,0.0,0.000013,0.000000,0.000000,0.0,0.000009,0.000000
4,0.064235,0.032112,0.000344,0.000301,0.000735,0.000295,0.000476,0.000313,0.000271,0.000187,...,0.000000,0.000000,0.000006,0.0,0.000006,0.000000,0.000000,0.0,0.000000,0.000000


In [8]:
df.shape

(60, 785)

Now we split the data into the label vector \begin{equation}\mathbf{y}\end{equation} and a training data matrix \begin{equation}\mathbf{X}\end{equation}. But first, we shuffle the df and split it into a training and a test set.

Moreover, it is necessary for many machine learning tasks to standardise the data. Our aim is for each feature to be represented by a column vector in which values have zero mean and unit variance.

In [9]:
def normalise_point(datapoint, mean, std):
    """
    normalise a datapoint to zero mean and unit variance.
    :param datapoint: value as a float
    :param mean: mean of data vector x
    :param std: standard deviation of data vector x
    :return: normalised datapoint (float)
    """
    return (datapoint - mean)/std

def normalise_matrix(matrix):
    """
    normalises the data matrix
    :param matrix: input matrix
    :return: normalised matrix
    """
    train_normalised = matrix.copy()
    
    for col in matrix:
        try:
            mean = matrix[col].mean()
            std = matrix[col].std()
            for index, item in enumerate(matrix[col]):
                train_normalised.loc[index, col] = normalise_point(item, mean, std)
        except ZeroDivisionError:
            train_normalised.loc[index, col] = 0.0
        except TypeError:
            continue
    return train_normalised

In [10]:
df_norm = normalise_matrix(df)

Split the data into a train set and a test set

In [11]:
train = df_norm.sample(frac=0.9, random_state=seed)
test = df_norm.drop(train.index)

Define the different sets

In [12]:
# for training
y_train = train.y
X_train = train.drop('y', axis=1)

# for testing
y_test = test.y
X_test = test.drop('y', axis=1)

Check the shapes

In [13]:
print('Training samples shape: ', X_train.shape)
print('Training labels shape: ', y_train.shape)
print('Test samples shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

Training samples shape:  (54, 784)
Training labels shape:  (54,)
Test samples shape:  (6, 784)
Test labels shape:  (6,)


We should binarise our labels, although libraries like sklearn can also deal with non-numeric data

In [14]:
from sklearn import preprocessing

lb = preprocessing.LabelBinarizer()
lb.fit(['en', 'fr', 'de', 'it', 'es', 'fi'])

LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)

In [15]:
lb.classes_

array(['de', 'en', 'es', 'fi', 'fr', 'it'], dtype='<U2')

We do this for both our training and test labels:

In [16]:
y_train = lb.transform(y_train)
y_test = lb.transform(y_test)

Labels are now one-hot encoded:

In [17]:
y_train[0]

array([0, 0, 0, 1, 0, 0])

We almost have everything now. However, we need to take care of the bias and the weight matrix. The hypothesis ŷ is given by:
\begin{equation}
\mathbf{\hat{y}}=\mathbf{x}\cdot\mathbf{W}+\mathbf{b}
\end{equation}
We can achieve this by appending 1 to each feature vector x, and the whole weight vector b to the weight matrix W. This is called the bias trick. Note that the dimensions of X_train change, and that the weight matrix W will have match the dimensions (same number of rows as X has columns).

In [18]:
bias_vector = np.ones([X_train.shape[0], 1])
X_train['bias'] = bias_vector
X_train

,$$,$_,$a,$b,$c,$d,$e,$f,$g,$h,...,zr,zs,zt,zu,zv,zw,zx,zy,zz,bias
17,0.416705,0.724450,0.415587,-0.679809,-0.973339,-0.650477,-0.396736,-0.943586,-0.864334,0.838599,...,-0.542996,-0.498490,-0.428225,-0.440930,-0.281712,-0.418136,-0.275911,-0.448815,-0.452241,1.0
5,1.120987,0.754591,-0.134707,-0.331671,-0.658595,-0.085424,-0.824043,-0.596442,-0.633121,-0.322912,...,-0.542996,-0.498490,-0.335276,-0.443910,-0.281712,-0.418136,-0.275911,0.247255,-0.426881,1.0
33,-0.325057,-0.583721,-1.236540,1.883605,-0.211154,-0.116536,-0.431807,-0.030221,0.358247,0.851821,...,0.710846,1.222124,2.205323,2.651505,-0.281712,2.055032,-0.275911,0.646854,-0.332487,1.0
48,-1.079415,0.965113,0.604424,-0.772101,-0.413109,-0.474101,-0.162789,-0.937614,-0.877095,-1.127295,...,-0.542996,-0.498490,-0.428225,-0.453582,-0.281712,-0.418136,-0.275911,-0.448815,1.417154,1.0
58,-1.162065,-1.478148,1.661925,-0.693171,0.422842,-0.342607,-0.757701,4.229579,1.266662,-1.002757,...,-0.542996,-0.498490,-0.428225,-0.412311,-0.281712,-0.418136,-0.275911,-0.448815,-0.452241,1.0
2,0.480037,-0.068256,-1.016916,-0.317620,-0.651997,-0.889553,-0.865333,-0.604453,-0.916012,-0.876075,...,-0.089174,-0.498490,-0.428225,-0.433836,0.414885,-0.404149,-0.275911,-0.052242,-0.452241,1.0
36,-0.419297,-0.628931,-0.256807,0.941286,-0.465767,0.939080,-0.617309,0.122219,0.616054,1.578355,...,2.391500,0.508242,1.455085,2.812389,1.970446,1.933324,-0.275911,1.260723,-0.405529,1.0
54,0.084960,-0.116761,0.369967,-0.562680,0.821571,-0.558429,0.326681,-0.429021,0.237262,-0.814364,...,-0.542996,0.051763,-0.428225,-0.450289,-0.281712,-0.418136,-0.275911,-0.448815,-0.375646,1.0
46,-0.869830,1.050150,0.771172,-0.119695,-0.119732,0.866934,-0.415106,-0.356439,-0.508251,-0.346814,...,0.986253,-0.498490,-0.428225,-0.438480,-0.281712,-0.418136,-0.275911,-0.003370,1.251760,1.0
51,-0.584746,-0.008377,-0.342159,-0.745939,-0.604093,-0.743701,-0.442223,-0.326390,-0.867180,-1.005471,...,-0.542996,-0.498490,-0.428225,-0.463310,-0.281712,-0.418136,2.072618,-0.448815,4.001936,1.0


In [19]:
# initialise weight matrix with small weights

np.random.seed(seed=200)

W = np.random.randn(X_train.shape[1], len(lb.classes_)) * 0.0001
#W = np.zeros([X.shape[1], len(lb.classes_)])

We see that the dimensions are right. The dot product of a specific row from X_train and the weight matrix W constitutes a forward pass and calculates the score for each class.

In [20]:
W.shape

(785, 6)

In [21]:
X_train.shape

(54, 785)

In [22]:
X_train[5:6].dot(W)

,0,1,2,3,4,5
2,-0.002753,0.001105,0.000354,-0.000783,0.003842,-0.000011


We see that the values for the highest score of the dot product is not the score of the true label. Our aim is to change this by implementing a support vector classifier.

In [23]:
X_train[5:6].dot(W).max(axis=1)

2    0.003842
dtype: float64

In [24]:
X_train[5:6].dot(W)[y_train[5:6].argmax()]

2    0.001105
Name: 1, dtype: float64

Important: we follow kind of a naive implementation. The aim is to be able to understand what is going on!

In order to quantify how good (or how bad) our weight matrix W can predict the data in our training set, we need to implement a loss function. Here we take a go at the hinge loss, which tries to predict the correct class with a margin of at least one to all other classes (or in this case, like presented in Goldberg, to the class which does not equal the true class, but which scores highest). In my understanding, this is a one-vs-one approach (true class vs. class with highest score (but doesn't equal the true class)).

In [116]:
def hinge_loss(x, y, W, index):
    """
    Calculates the loss of a single data point by taking the prediction of the correct value and the the prediction of
    the value of next highest score, following Crammer and Singer (2001)
    :param x: sample point x as a vector
    :param y: correct label y for x as a vector
    :param W: weight matrix
    :param index: column index of data matrix X
    :return: loss
    """
    loss = 0
    y_index = y[index].argmax()
    y_value = x.dot(W)[y_index]
    y_hat_max_value = np.delete(x.dot(W), y_index).max()
    #for j in range(0, y.shape[1]):  # in case we wanted to classify against all other classes (one-vs-all) --> currently one-vs-one
        #if j == y_index:
            #continue
    loss += max(0, 1 - (y_value - y_hat_max_value))
    return loss

With matrix multiplication, we could get all the scores at once. In the following, however, we focus on an approach which takes sample by sample and calculates the loss and the gradients.

In [26]:
scores = X_train.dot(W)  # simple matrix multiplication to get all scores

In [27]:
scores

,0,1,2,3,4,5
17,-0.007375,-0.005236,-0.004241,-0.006293,0.000811,0.003541
5,-0.005800,-0.006748,0.001192,-0.000476,0.000874,-0.001014
33,0.003552,0.001369,-0.001093,-0.000351,-0.000555,0.003478
48,0.002507,0.002691,0.002067,0.000304,-0.000161,-0.002081
58,0.000668,-0.000811,0.006183,0.002916,-0.002477,0.000341
2,-0.002753,0.001105,0.000354,-0.000783,0.003842,-0.000011
36,0.000501,-0.005079,-0.002875,0.004025,0.001413,-0.000616
54,-0.001154,0.004904,0.003003,0.005856,-0.001277,0.004642
46,-0.002021,0.000087,0.001625,-0.000911,-0.001955,-0.004749
51,-0.000868,0.004471,0.001184,0.001439,0.001762,-0.004693


In [120]:
def gradient(X, y, W):
    """
    compute the gradient
    :param X: data matrix (train) 
    :param y: the corresponding 
    :param W: weight matrix
    :return: loss and Jacobian dW with all gradients
    """
    dW = np.zeros(W.shape)
    
    total_loss = 0.0
    
    for index, x in enumerate(X.as_matrix()):
        y_index = y[index].argmax()
        y_value = x.dot(W)[y_index]
        y_hat_max_value = np.delete(x.dot(W), y_index).max()
        loss = max(0, 1 - (y_value - y_hat_max_value))
        total_loss += loss
        y_hat_max_index = np.delete(x.dot(W), y_index).argmax() + 1
        if loss > 0:  # not sure whether we need this if statement
            dW[:, y_hat_max_index] += x.transpose()
            dW[:, y_index] -= x.transpose()
            
    return total_loss, dW
    

In [121]:
def gradient_descent(X, y, W, eta, steps):
    """
    Perform gradient descent for a number of times with a fixed learning rate eta
    :param X: data matrix
    :param y: labels
    :param W: weight matrix
    :param eta: learning rate
    :param steps: number of times gradient descent should be performed
    :return: learned representation matrix W_learned
    """
    W_learned = W.copy()
    
    for step in range(0, steps):
        loss, dW = gradient(X, y, W_learned)
        print(loss)
        W_learned = W_learned - eta * dW
        
    return W_learned
    

In [122]:
W_star = gradient_descent(X_train, y_train, W, eta=0.001, steps=10)

54.206118975295375
21.486670396698702
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


### Testing
Let's test if our learned representation of the data is any good at classifying the data in the test set. Of course we need the bias in our test set as well!

In [83]:
bias_vector_test = np.ones([X_test.shape[0], 1])
X_test['bias'] = bias_vector_test

In [84]:
for index, x in enumerate(X_test.dot(W_star).as_matrix()):
    pred = x.argmax()
    true_label = y_test[index].argmax()
    print(pred, true_label)

1 1
3 3
3 3
4 4
0 0
5 5


Not too bad! But Goldberg mentioned something about regularisation, so we should take this into account as well!

In [113]:
def gradient_reg(X, y, W, lam):
    """
    compute the gradient
    :param X: data matrix (train) 
    :param y: the corresponding 
    :param W: weight matrix
    :param lam: reguliser lambda
    :return: Jacobian dW with all gradients
    """
    dW = np.zeros(W.shape)
    
    total_loss = 0.0
    
    for index, x in enumerate(X.as_matrix()):
        y_index = y[index].argmax()
        y_value = x.dot(W)[y_index]
        y_hat_max_value = np.delete(x.dot(W), y_index).max()
        loss = max(0, 1 - (y_value - y_hat_max_value)) + lam * np.linalg.norm(W, 2)
        total_loss += loss
        y_hat_max_index = np.delete(x.dot(W), y_index).argmax() + 1
        if loss > 0:  # not sure whether we need this if statement
            dW[:, y_hat_max_index] += x.transpose()
            dW[:, y_index] -= x.transpose()
        
    dW += 2 * lam * W
            
    return total_loss, dW

def gradient_descent_reg(X, y, W, eta, steps):
    """
    Perform gradient descent for a number of times with a fixed learning rate eta
    :param X: data matrix
    :param y: labels
    :param W: weight matrix
    :param eta: learning rate
    :param steps: number of times gradient descent should be performed
    :return: learned representation matrix W_learned
    """
    W_learned = W.copy()
    
    for step in range(0, steps):
        loss, dW = gradient_reg(X, y, W_learned, 10)
        print(loss)
        W_learned = W_learned - eta * dW
        
    return W_learned
    

In [114]:
W_star_reg = gradient_descent_reg(X_train, y_train, W, eta=0.001, steps=10)

55.78230906772709
203.21634783496432
283.08255355815544
394.5485516212814
524.8345169607236
635.7249949392508
748.132597614688
864.2076556461765
991.654462158203
1086.838496318409


In [115]:
for index, x in enumerate(X_test.dot(W_star_reg).as_matrix()):
    pred = x.argmax()
    true_label = y_test[index].argmax()
    print(pred, true_label)

1 1
3 3
3 3
4 4
0 0
5 5


If we look at the two different weight matrices (one regularised, the other not), we notice the following:

In [109]:
W_star[0:5]

array([[-0.0037736 ,  0.00624098, -0.00675007,  0.01021503, -0.00699125,
         0.00118419],
       [-0.00829818,  0.00381192, -0.01537289,  0.00282321,  0.0122733 ,
         0.00493589],
       [-0.00743025, -0.00597328,  0.01602757, -0.00920754,  0.01962092,
        -0.01269763],
       [ 0.01578216, -0.00314716, -0.00245102, -0.00663084,  0.0035217 ,
        -0.0071784 ],
       [-0.00068613, -0.00424234,  0.00507315, -0.00952214,  0.02428383,
        -0.0148356 ]])

In [110]:
W_star_reg[0:5]

array([[-0.03482627,  0.04752469, -0.04082376,  0.06112236, -0.0103456 ,
        -0.02253813],
       [-0.07915388,  0.03398947, -0.08130804, -0.01123183,  0.08562072,
         0.05224025],
       [-0.07123135, -0.05690076,  0.07710793, -0.06656845,  0.11861532,
        -0.00071539],
       [ 0.15165281, -0.06477741, -0.02128677, -0.02910499,  0.00058112,
        -0.03715841],
       [-0.00627308, -0.04574701,  0.01419929, -0.09809548,  0.15285981,
        -0.01687954]])

## By the way ...
### In scikit-learn it's much easier to implement this :-)


In [39]:
from sklearn.svm import LinearSVC
clf = LinearSVC(random_state=0, multi_class='crammer_singer', loss='hinge')
clf.fit(X_train, train.y)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='hinge', max_iter=1000,
     multi_class='crammer_singer', penalty='l2', random_state=0,
     tol=0.0001, verbose=0)

In [40]:
clf.predict(X_test)

array(['en', 'fi', 'fi', 'fr', 'de', 'it'], dtype=object)

In [41]:
test.y

4     en
12    fi
16    fi
26    fr
41    de
42    it
Name: y, dtype: object

We see that with our naive implementation, we do not much worse than with scikit's. scikit's implementation is of course much more elaborate and uses the vectorised operation and possibly other optimisation techniques in order to make its SVM (or SVC) better.